# 导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./huanhuan.json') # 注意修改
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

# 处理数据集

In [4]:
tokenizer = AutoTokenizer.from_pretrained('/model/ModelScope/ZhipuAI/GLM-4.5-Air')

In [7]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你好"},
    {"role": "assistant", "content": "你好，我是一个AI助手"},
    {"role": "user", "content": "不错～"},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
print(text)

[gMASK]<sop><|system|>
You are a helpful assistant.<|user|>
你好<|assistant|>
<think></think>
你好，我是一个AI助手<|user|>
不错～<|assistant|>


In [6]:
def process_func(example):
    MAX_LENGTH = 1024 # 设置最大序列长度为1024个token
    input_ids, attention_mask, labels = [], [], [] # 初始化返回值
    # 适配chat_template
    instruction = tokenizer(
        f"[gMASK]<sop><|system|>\n现在你要扮演皇帝身边的女人--甄嬛" 
        f"<|user|>\n{example['instruction'] + example['input']}"  
        f"<|assistant|>\n<think></think>\n",  
        add_special_tokens=False   
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    # 将instructio部分和response部分的input_ids拼接，并在末尾添加eos token作为标记结束的token
    input_ids = instruction["input_ids"] + response["input_ids"]
    # 注意力掩码，表示模型需要关注的位置
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 对于instruction，使用-100表示这些位置不计算loss（即模型不需要预测这部分）
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:  # 超出最大序列长度截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3729/3729 [00:01<00:00, 2785.39 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [8]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))

[gMASK]<sop><|system|>
现在你要扮演皇帝身边的女人--甄嬛<|user|>
小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——<|assistant|>
<think></think>
嘘——都说许愿说破是不灵的。


In [9]:
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"]))))

你们俩话太多了，我该和温太医要一剂药，好好治治你们。


# 创建模型

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained('/model/ModelScope/ZhipuAI/GLM-4.5-Air', device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards: 100%|██████████| 47/47 [00:45<00:00,  1.04it/s]
Some weights of the model checkpoint at /model/ModelScope/ZhipuAI/GLM-4.5-Air were not used when initializing Glm4MoeForCausalLM: ['model.layers.46.eh_proj.weight', 'model.layers.46.embed_tokens.weight', 'model.layers.46.enorm.weight', 'model.layers.46.hnorm.weight', 'model.layers.46.input_layernorm.weight', 'model.layers.46.mlp.experts.0.down_proj.weight', 'model.layers.46.mlp.experts.0.gate_proj.weight', 'model.layers.46.mlp.experts.0.up_proj.weight', 'model.layers.46.mlp.experts.1.down_proj.weight', 'model.layers.46.mlp.experts.1.gate_proj.weight', 'model.layers.46.mlp.experts.1.up_proj.weight', 'model.layers.46.mlp.experts.10.down_proj.weight', 'model.layers.46.mlp.experts.10.gate_proj.weight', 'model.layers.46.mlp.experts.10.up_proj.weight', 'model.layers.46.mlp.experts.100.down_proj.weight', 'model.layers.46.mlp.experts.100.gate_proj.weight', 'model.layers.46.mlp.experts.100.up_proj.weight', 'model.layers.

Glm4MoeForCausalLM(
  (model): Glm4MoeModel(
    (embed_tokens): Embedding(151552, 4096, padding_idx=151329)
    (layers): ModuleList(
      (0): Glm4MoeDecoderLayer(
        (self_attn): Glm4MoeAttention(
          (q_proj): Linear(in_features=4096, out_features=12288, bias=True)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=True)
          (o_proj): Linear(in_features=12288, out_features=4096, bias=False)
        )
        (mlp): Glm4MoeMLP(
          (gate_proj): Linear(in_features=4096, out_features=10944, bias=False)
          (up_proj): Linear(in_features=4096, out_features=10944, bias=False)
          (down_proj): Linear(in_features=10944, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Glm4MoeRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Glm4MoeRMSNorm((4096,), eps=1e-05)
      )
      (1-45): 45 x Glm4MoeDecoderLayer

In [11]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [12]:
model.dtype

torch.bfloat16

# lora 

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules={'up_proj', 'q_proj', 'down_proj', 'o_proj', 'gate_proj', 'k_proj', 'v_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/model/ModelScope/ZhipuAI/GLM-4.5-Air', revision=None, inference_mode=False, r=8, target_modules={'up_proj', 'q_proj', 'down_proj', 'o_proj', 'gate_proj', 'k_proj', 'v_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [15]:
model.print_trainable_parameters()

trainable params: 783,005,184 || all params: 107,635,250,688 || trainable%: 0.7275


# 配置训练参数

In [16]:
args = TrainingArguments(
    output_dir="./output/glm45_air_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=2,
    num_train_epochs=3,
    save_steps=10, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

In [17]:
import swanlab
from swanlab.integration.transformers import SwanLabCallback

swanlab.login(api_key='BIYVGq2rfWmD9sFMCehUG', save=True)

# 实例化SwanLabCallback
swanlab_callback = SwanLabCallback(
    project="self-llm", 
    experiment_name="glm45_air_lora_experiment"
)

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

swanlab: Tracking run with swanlab version 0.6.7

swanlab: Run data will be saved locally in /workspace/swanlog/run-20250729_034415-g2v55jfocwebf6qhaucu3

swanlab: 👋 Hi kmno4,welcome to swanlab!

swanlab: Syncing run glm45_air_lora_experiment to the cloud

swanlab: 🏠 View project at https://swanlab.cn/@kmno4/self-llm

swanlab: 🚀 View run at https://swanlab.cn/@kmno4/self-llm/runs/g2v55jfocwebf6qhaucu3

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
2,3.537200
4,3.383800
6,2.945900
8,3.113100
10,3.166800
12,2.749700
14,3.000500
16,2.643000
18,2.921100
20,3.173600


# 合并加载模型

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/model/ModelScope/ZhipuAI/GLM-4.5-Air'
lora_path = '/workspace/output/glm45_air_lora/checkpoint-20' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 47/47 [00:46<00:00,  1.02it/s]
Some weights of the model checkpoint at /model/ModelScope/ZhipuAI/GLM-4.5-Air were not used when initializing Glm4MoeForCausalLM: ['model.layers.46.eh_proj.weight', 'model.layers.46.embed_tokens.weight', 'model.layers.46.enorm.weight', 'model.layers.46.hnorm.weight', 'model.layers.46.input_layernorm.weight', 'model.layers.46.mlp.experts.0.down_proj.weight', 'model.layers.46.mlp.experts.0.gate_proj.weight', 'model.layers.46.mlp.experts.0.up_proj.weight', 'model.layers.46.mlp.experts.1.down_proj.weight', 'model.layers.46.mlp.experts.1.gate_proj.weight', 'model.layers.46.mlp.experts.1.up_proj.weight', 'model.layers.46.mlp.experts.10.down_proj.weight', '

In [ ]:
prompt = "你是谁？"
inputs = tokenizer.apply_chat_template(
                                    [{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                    add_generation_prompt=True,
                                    tokenize=True,
                                    return_tensors="pt",
                                    return_dict=True,
                                    enable_thinking=False
                                )

inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
}

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/miniconda3/envs/py310/lib/python3.10/site-packages/transformers/generation/utils.py:2505: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
